<h1>EDA to get buyer and seller valuation</h1>

In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [50]:
# Load excel file into a DataFrame
df = pd.read_excel("../datasets/resalepricesadjusted.xlsx")

print('before')
df.head()
df['Neighborhood score'] = df['Neighborhood score'].astype(float)
df['flat_type'] = df['flat_type'].astype(float)
df['remaining_lease'] = df['remaining_lease'].astype(float)
df['resale_price'] = df['resale_price'].astype(float)
df['mean price'] = df['mean price'].astype(float)
df['ln_resale_price'] = df['ln_resale_price'].astype(float)
df['ln_mean_price'] = df['ln_mean_price'].astype(float)



before


In [51]:
print('after')
df.head()

after


,month,town,Neighborhood score,flat_type,lease_commence_date,remaining_lease,resale_price,mean price,ln_resale_price,ln_mean_price
0,2017-01,ANG MO KIO,4.0,2.0,1979,61.0,232000.0,413971.21,12.35,12.933552
1,2017-01,ANG MO KIO,4.0,3.0,1978,60.0,250000.0,413971.21,12.43,12.933552
2,2017-01,ANG MO KIO,4.0,3.0,1980,62.0,262000.0,413971.21,12.48,12.933552
3,2017-01,ANG MO KIO,4.0,3.0,1980,62.0,265000.0,413971.21,12.49,12.933552
4,2017-01,ANG MO KIO,4.0,3.0,1980,62.0,265000.0,413971.21,12.49,12.933552


In [52]:
# Assuming df is your DataFrame
# Selecting only the specified columns from the dataframe
X_columns = ['ln_mean_price', 'Neighborhood score', 'flat_type', 'remaining_lease']
X = df[X_columns]

# Find rows with NaN values in the specified columns
rows_with_nan = X[X.isna().any(axis=1)]

# Display rows with NaN values
print(rows_with_nan)

Empty DataFrame
Columns: [ln_mean_price, Neighborhood score, flat_type, remaining_lease]
Index: []


In [53]:
X_columns = ['ln_mean_price','Neighborhood score','flat_type','remaining_lease']
# Selecting only the specified columns from the dataframe
X = df[X_columns]

# Splitting the dataset into features and target variable
y = df['ln_resale_price'] # Target variable

# Splitting the dataset into the training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training the model
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [54]:
# Making predictions
y_pred = model.predict(X_test)

In [55]:
# Model evaluation
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

# Coefficients of the model
coefficients = pd.DataFrame(model.coef_, X.columns, columns=['Coefficient'])
print("\nCoefficients:")
print(coefficients)

# Intercept of the model
print("\nIntercept:", model.intercept_)

Mean Squared Error: 0.027084766194313693
R-squared: 0.7589335322721764

Coefficients:
                    Coefficient
ln_mean_price          0.971453
Neighborhood score     0.011752
flat_type              0.231954
remaining_lease        0.006445

Intercept: -1.1636462846180269


<h1>Formula</h1>

$$
\text{resale\_price} = e^{(0.971 \times \ln(\text{mean\_price})) + (0.012 \times \text{Neighborhood score}) + (0.232 \times \text{number\_of\_rooms}) + (0.006 \times \text{remaining\_lease}) - 1.164}
$$

<h1>Seller Valuation</h1>

In [56]:
X_columns = ['ln_mean_price', 'remaining_lease']
# Selecting only the specified columns from the dataframe
X = df[X_columns]

# Splitting the dataset into features and target variable
y = df['ln_resale_price'] # Target variable

# Splitting the dataset into the training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training the model
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [57]:
y_pred = model.predict(X_test)

In [58]:
# Model evaluation
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

# Coefficients of the model
coefficients = pd.DataFrame(model.coef_, X.columns, columns=['Coefficient'])
print("\nCoefficients:")
print(coefficients)

# Intercept of the model
print("\nIntercept:", model.intercept_)

Mean Squared Error: 0.06919588307356668
R-squared: 0.3841258590094727

Coefficients:
                 Coefficient
ln_mean_price       0.998532
remaining_lease     0.009271

Intercept: -0.7110911321760103


<h1>Formula</h1>

$$
\text{resale\_price} = e^{(0.998 \times \ln(\text{mean\_price}) + 0.009 \times \text{remaining\_lease}) - 0.711}
$$